<h1 align=center><font size = 5>Regression Models with Keras</font></h1>

## Introduction

As we discussed in the videos, despite the popularity of more powerful libraries such as PyToch and TensorFlow, they are not easy to use and have a steep learning curve. So, for people who are just starting to learn deep learning, there is no better library to use other than the Keras library. 

Keras is a high-level API for building deep learning models. It has gained favor for its ease of use and syntactic simplicity facilitating fast development. As you will see in this lab and the other labs in this course, building a very complex deep learning network can be achieved with Keras with only few lines of code. You will appreciate Keras even more, once you learn how to build deep models using PyTorch and TensorFlow in the other courses.

So, in this lab, you will learn how to use the Keras library to build a regression model.


<h2>Regression Models with Keras</h2>

<h3>Objective for this Notebook<h3>    
<h5> 1. How to use the Keras library to build a regression model.</h5>
<h5> 2. Download and Clean dataset </h5>
<h5> 3. Build a Neural Network </h5>
<h5> 4. Train and Test the Network. </h5>     


## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>
    
1. <a href="#item31">Upload and Clean Dataset</a>  
2. <a href="#item32">Import Keras</a>  
3. <a href="#item33">Build a Neural Network</a>  
4. <a href="#item34">Train and Test the Network</a>  

</font>
</div>


## Upload and Clean Dataset

Let's start by importing the <em>pandas</em> and the Numpy libraries.

In [3]:
import pandas as pd
import numpy as np

We will be playing around with the same dataset that we used in the videos.

<strong>The dataset is about the compressive strength of different samples of concrete based on the volumes of the different ingredients that were used to make them. Ingredients include:</strong>

<strong>1. Cement</strong>

<strong>2. Blast Furnace Slag</strong>

<strong>3. Fly Ash</strong>

<strong>4. Water</strong>

<strong>5. Superplasticizer</strong>

<strong>6. Coarse Aggregate</strong>

<strong>7. Fine Aggregate</strong>


Let's upload the data and read it into a <em>pandas</em> dataframe.

In [4]:
concrete_data = pd.read_csv('concrete_data.csv')
concrete_data.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


So the first concrete sample has 540 cubic meter of cement, 0 cubic meter of blast furnace slag, 0 cubic meter of fly ash, 162 cubic meter of water, 2.5 cubic meter of superplaticizer, 1040 cubic meter of coarse aggregate, 676 cubic meter of fine aggregate. Such a concrete mix which is 28 days old, has a compressive strength of 79.99 MPa. 


#### Let's check how many data points we have.

In [5]:
concrete_data.shape

(1030, 9)

So, there are approximately 1000 samples to train our model on. Because of the few samples, we have to be careful not to overfit the training data.


Let's check the dataset for any missing values.

In [6]:
concrete_data.describe()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000
mean,281.167864,73.895825,54.188350,181.567282,6.204660,972.918932,773.580485,45.662136,35.817961
std,104.506364,86.279342,63.997004,21.354219,5.973841,77.753954,80.175980,63.169912,16.705742
min,102.000000,0.000000,0.000000,121.800000,0.000000,801.000000,594.000000,1.000000,2.330000
25%,192.375000,0.000000,0.000000,164.900000,0.000000,932.000000,730.950000,7.000000,23.710000
50%,272.900000,22.000000,0.000000,185.000000,6.400000,968.000000,779.500000,28.000000,34.445000
75%,350.000000,142.950000,118.300000,192.000000,10.200000,1029.400000,824.000000,56.000000,46.135000
max,540.000000,359.400000,200.100000,247.000000,32.200000,1145.000000,992.600000,365.000000,82.600000


In [7]:
concrete_data.isnull().sum()

Cement                0
Blast Furnace Slag    0
Fly Ash               0
Water                 0
Superplasticizer      0
Coarse Aggregate      0
Fine Aggregate        0
Age                   0
Strength              0
dtype: int64

The data looks very clean and is ready to be used to build our model.


#### Split data into predictors and target


The target variable in this problem is the concrete sample strength. Therefore, our predictors will be all the other columns.


In [8]:
concrete_data_columns = concrete_data.columns

predictors = concrete_data[concrete_data_columns[concrete_data_columns != 'Strength']] # all columns except Strength
target = concrete_data['Strength'] # Strength column

Let's do a quick sanity check of the predictors and the target dataframes.


In [9]:
predictors.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360


In [10]:
target.head()

0    79.99
1    61.89
2    40.27
3    41.05
4    44.30
Name: Strength, dtype: float64

Finally, the last step is to normalize the data by substracting the mean and dividing by the standard deviation.


In [11]:
predictors_norm = (predictors - predictors.mean()) / predictors.std()
predictors_norm.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,0.862735,-1.217079,-0.279597
1,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,1.055651,-1.217079,-0.279597
2,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,3.551340
3,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,5.055221
4,-0.790075,0.678079,-0.846733,0.488555,-1.038638,0.070492,0.647569,4.976069


Let's save the number of predictors to _n_cols_ since we will need this number when building our network.

In [12]:
n_cols = predictors_norm.shape[1] # number of predictors

## Import Keras

Recall from the videos that Keras normally runs on top of a low-level library such as TensorFlow. This means that to be able to use the Keras library, you will have to install TensorFlow first and when you import the Keras library, it will be explicitly displayed what backend was used to install the Keras library. In CC Labs, we used TensorFlow as the backend to install Keras, so it should clearly print that when we import Keras.


#### Let's go ahead and import the Keras library

In [13]:
import tensorflow as tf

In [14]:
from tensorflow import keras

In [17]:
from tensorflow.keras.models import Sequential

In [18]:
from tensorflow.keras.layers import Dense

## Build a Neural Network

Let's define a function that defines our regression model for us so that we can conveniently call it to create our model.


In [19]:
# define regression model
def regression_model():
    # create model
    model = Sequential()
    model.add(Dense(50, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(1))
    
    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

The above function create a model that has two hidden layers, each of 50 hidden units.

## Train and Test the Network

Let's call the function now to create our model.

In [20]:
# build the model
model = regression_model()

Next, we will train and test the model at the same time using the _fit_ method. We will leave out 30% of the data for validation and we will train the model for 100 epochs.


In [26]:
# fit the model
model.fit(predictors_norm, target, validation_split=0.3, epochs=200, verbose=2)

Epoch 1/200
23/23 - 0s - loss: 6.3162 - val_loss: 141.3258
Epoch 2/200
23/23 - 0s - loss: 6.1502 - val_loss: 132.3132
Epoch 3/200
23/23 - 0s - loss: 6.1807 - val_loss: 135.5434
Epoch 4/200
23/23 - 0s - loss: 6.1411 - val_loss: 140.5093
Epoch 5/200
23/23 - 0s - loss: 6.3675 - val_loss: 141.7892
Epoch 6/200
23/23 - 0s - loss: 6.3867 - val_loss: 135.1261
Epoch 7/200
23/23 - 0s - loss: 6.1805 - val_loss: 131.4818
Epoch 8/200
23/23 - 0s - loss: 6.1998 - val_loss: 130.2957
Epoch 9/200
23/23 - 0s - loss: 6.2870 - val_loss: 131.1382
Epoch 10/200
23/23 - 0s - loss: 6.4120 - val_loss: 138.7048
Epoch 11/200
23/23 - 0s - loss: 6.4253 - val_loss: 128.6083
Epoch 12/200
23/23 - 0s - loss: 7.0689 - val_loss: 139.8785
Epoch 13/200
23/23 - 0s - loss: 6.2372 - val_loss: 138.6135
Epoch 14/200
23/23 - 0s - loss: 6.3911 - val_loss: 138.7412
Epoch 15/200
23/23 - 0s - loss: 6.4514 - val_loss: 141.0912
Epoch 16/200
23/23 - 0s - loss: 6.2065 - val_loss: 137.9057
Epoch 17/200
23/23 - 0s - loss: 6.3362 - val_loss

Epoch 138/200
23/23 - 0s - loss: 5.7996 - val_loss: 140.8552
Epoch 139/200
23/23 - 0s - loss: 5.9254 - val_loss: 136.9077
Epoch 140/200
23/23 - 0s - loss: 5.6671 - val_loss: 135.0126
Epoch 141/200
23/23 - 0s - loss: 5.6823 - val_loss: 135.7192
Epoch 142/200
23/23 - 0s - loss: 5.7440 - val_loss: 140.3732
Epoch 143/200
23/23 - 0s - loss: 5.6726 - val_loss: 143.7360
Epoch 144/200
23/23 - 0s - loss: 6.2599 - val_loss: 142.2855
Epoch 145/200
23/23 - 0s - loss: 5.9800 - val_loss: 142.7024
Epoch 146/200
23/23 - 0s - loss: 6.3992 - val_loss: 139.3969
Epoch 147/200
23/23 - 0s - loss: 5.7354 - val_loss: 136.8497
Epoch 148/200
23/23 - 0s - loss: 5.6367 - val_loss: 136.9370
Epoch 149/200
23/23 - 0s - loss: 5.7413 - val_loss: 134.4087
Epoch 150/200
23/23 - 0s - loss: 5.9329 - val_loss: 141.4455
Epoch 151/200
23/23 - 0s - loss: 5.6351 - val_loss: 138.0750
Epoch 152/200
23/23 - 0s - loss: 5.6263 - val_loss: 140.1592
Epoch 153/200
23/23 - 0s - loss: 5.7022 - val_loss: 136.6949
Epoch 154/200
23/23 - 0s

<strong>You can refer to this [link](https://keras.io/models/sequential?cm_mmc=Email_Newsletter-_-Developer_Ed%2BTech-_-WW_WW-_-SkillsNetwork-Courses-IBMDeveloperSkillsNetwork-DL0101EN-SkillsNetwork-20718188&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ&cm_mmc=Email_Newsletter-_-Developer_Ed%2BTech-_-WW_WW-_-SkillsNetwork-Courses-IBMDeveloperSkillsNetwork-DL0101EN-SkillsNetwork-20718188&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ&cm_mmc=Email_Newsletter-_-Developer_Ed%2BTech-_-WW_WW-_-SkillsNetwork-Courses-IBMDeveloperSkillsNetwork-DL0101EN-SkillsNetwork-20718188&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ) to learn about other functions that you can use for prediction or evaluation.</strong>
